# Camada trusted
## Aplicando tratamentos nos dados

Importando bibliotecas necessárias

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)

Lendo dados da camada raw

In [5]:
trzd_olist_order_reviews = pd.read_parquet(
    path='../dataset/raw/rwzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Filtrando por avaliações que tenham ao menos uma mensagem, mas não necessariamente o título

In [6]:
trzd_olist_order_reviews.dropna(thresh=6, inplace=True)

Filtrando avaliações que comentaram sobre entregas nos comentários

In [7]:
deliver_message_mask = trzd_olist_order_reviews['review_comment_message'].str.contains(
    pat='entrega|entregar|entregue|prazo|recebi|recebido|chego|chegou|correio|correios|demora|demorou|demorar|atraso|atrasado|frete',
    case=False,
    regex=True,
    na=False,
)

Filtrando avaliações que comentaram sobre entregas no título

In [8]:
deliver_title_mask = trzd_olist_order_reviews['review_comment_title'].str.contains(
    pat='entrega|entregar|entregue|prazo|recebi|recebido|chego|chegou|correio|correios|demora|demorou|demorar|atraso|atrasado|frete',
    case=False,
    regex=True,
    na=False,
)

Criando máscara para identificar avaliações que comentaram sobre entregas tanto no título quanto nos comentários

In [9]:
deliver_mask = (deliver_message_mask | deliver_title_mask)

Removendo avaliações sobre entregas uma vez que o foco está nas avaliações de produtos

In [10]:
deliver_reviews_index = trzd_olist_order_reviews.loc[deliver_mask].index

In [11]:
trzd_olist_order_reviews.drop(
    index=deliver_reviews_index,
    inplace=True
)

In [12]:
trzd_olist_order_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
99967,0e7bc73fde6782891898ea71443f9904,bd78f91afbb1ecbc6124974c5e813043,4,👍,Aprovado!,2018-07-04,2018-07-05 00:25:13
99971,58be140ccdc12e8908ff7fd2ba5c7cb0,0ebf8e35b9807ee2d717922d5663ccdb,5,muito bom produto,"Ficamos muito satisfeitos com o produto, atende às nossas necessidades com muita facilidade. Recomendo!",2018-06-30,2018-07-02 23:09:35
99974,40743b46a0ee86375cedb95e82b78d75,3e93213bb8fdda91186b4018b2fe0030,5,OTIMA EMBALAGEM,None,2018-08-08,2018-08-08 16:56:16
99975,2ee221b28e5b6fceffac59487ed39348,f2d12dd37eaef72ed7b1186b2edefbcd,2,Foto enganosa,Foto muito diferente principalmente a graninha sintética muito rala...diferente da foto,2018-03-28,2018-05-25 01:23:26
99990,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,None,"O produto não foi enviado com NF, não existe venda sem NF, com certeza fico no aguardo do envio da NF podendo ser por e-mail.",2018-05-19,2018-05-20 21:51:06


In [13]:
trzd_olist_order_reviews.count()

,0
review_id,20929
order_id,20929
review_score,20929
review_comment_title,6242
review_comment_message,19335
review_creation_date,20929
review_answer_timestamp,20929


Salvando dados na camada trusted em formato parquet

In [15]:
trzd_olist_order_reviews.to_parquet(
    path='../dataset/trusted/trzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)